In [1]:
import pandas as pd
import numpy as np

In [2]:
string_data=pd.read_csv('9606.protein.links.v12.0.txt',sep=' ')
string_data["protein1"]=string_data["protein1"].str.split('.').str[1]
string_data["protein2"]=string_data["protein2"].str.split('.').str[1]
string_data

,protein1,protein2,combined_score
0,ENSP00000000233,ENSP00000356607,173
1,ENSP00000000233,ENSP00000427567,154
2,ENSP00000000233,ENSP00000253413,151
3,ENSP00000000233,ENSP00000493357,471
4,ENSP00000000233,ENSP00000324127,201
...,...,...,...
13715399,ENSP00000501317,ENSP00000475489,195
13715400,ENSP00000501317,ENSP00000370447,158
13715401,ENSP00000501317,ENSP00000312272,226
13715402,ENSP00000501317,ENSP00000402092,169


### Convert the protein ids to gene ids

In [3]:
from gprofiler import GProfiler

# Initialize g:Profiler
def get_ensg_ids(ensp_ids, column_name):
    gp = GProfiler(return_dataframe=True)

    conversion_result = gp.convert(organism='hsapiens', query=ensp_ids, target_namespace='ENSG')
    conversion_result = conversion_result[["incoming","converted"]]
    conversion_result.columns = [column_name, column_name+"_ENSG"]

    return conversion_result


In [4]:
from pybiomart import Dataset

dataset = Dataset(name='hsapiens_gene_ensembl', host='http://www.ensembl.org')


df = dataset.query(attributes=['ensembl_gene_id', 'external_gene_name',"ensembl_peptide_id"])

In [5]:

mapper=df.dropna(subset=["Protein stable ID","Gene stable ID"]).set_index("Protein stable ID")["Gene stable ID"]
mapper
# fazer um map 

Protein stable ID
ENSP00000354687    ENSG00000198888
ENSP00000355046    ENSG00000198763
ENSP00000354499    ENSG00000198804
ENSP00000354876    ENSG00000198712
ENSP00000355265    ENSG00000228253
                        ...       
ENSP00000373370    ENSG00000143520
ENSP00000355866    ENSG00000143507
ENSP00000483812    ENSG00000143507
ENSP00000482935    ENSG00000143507
ENSP00000480008    ENSG00000143507
Name: Gene stable ID, Length: 123495, dtype: object

In [6]:
index_counts = mapper.index.value_counts()
index_counts

Protein stable ID
ENSP00000354687    1
ENSP00000473546    1
ENSP00000356998    1
ENSP00000357000    1
ENSP00000356999    1
                  ..
ENSP00000468644    1
ENSP00000474913    1
ENSP00000476009    1
ENSP00000468545    1
ENSP00000480008    1
Name: count, Length: 123495, dtype: int64

In [7]:
string_data["protein1"]=string_data["protein1"].map(mapper)
string_data["protein2"] = string_data["protein2"].map(mapper)



string_data

,protein1,protein2,combined_score
0,ENSG00000004059,ENSG00000116191,173
1,ENSG00000004059,ENSG00000137460,154
2,ENSG00000004059,ENSG00000131100,151
3,ENSG00000004059,ENSG00000105443,471
4,ENSG00000004059,ENSG00000156011,201
...,...,...,...
13715399,NaN,ENSG00000051825,195
13715400,NaN,ENSG00000182583,158
13715401,NaN,ENSG00000175155,226
13715402,NaN,ENSG00000164483,169


In [8]:
string_data.dropna(inplace=True,subset=["protein1","protein2"])
string_data.sort_values(by="combined_score",ascending=False,inplace=True)
string_data.drop_duplicates(subset=["protein1","protein2"],inplace=True)
string_data

,protein1,protein2,combined_score
10756603,ENSG00000232810,ENSG00000067182,999
5765613,ENSG00000186844,ENSG00000187223,999
10863475,ENSG00000132155,ENSG00000164924,999
10133976,ENSG00000100138,ENSG00000101161,999
3702726,ENSG00000164338,ENSG00000108651,999
...,...,...,...
4694853,ENSG00000138668,ENSG00000026508,150
4694879,ENSG00000138668,ENSG00000138029,150
4694918,ENSG00000138668,ENSG00000073578,150
11908652,ENSG00000149115,ENSG00000137337,150


In [9]:
string_data.to_csv("string_protein_links.txt",sep=' ',index=False)

In [7]:
import os
import glob

# Specify the folder path
folder_path = 'Clean_embeddings/'

# Use glob to get all file paths in the folder
file_paths = glob.glob(os.path.join(folder_path, '*'))
file_paths



['Clean_embeddings\\deepwalk_cbow_embedding.csv',
 'Clean_embeddings\\deepwalk_skipgram_embedding.csv',
 'Clean_embeddings\\first_order_line_embedding.csv',
 'Clean_embeddings\\glee_embedding.csv',
 'Clean_embeddings\\node2vec_cbow_embedding.csv',
 'Clean_embeddings\\node2vec_skipgram_embedding.csv',
 'Clean_embeddings\\second_order_line_embedding.csv']